## Tables and Data Source setup

In [1]:
using ODBC
using DataFrames
using DSWB
using Formatting
using URIParser

dsn = "dswb-natgeo" # Redshift esetTable(tableRt, tableType = "RESOURCE_TABLE")ndpoint
table = "beacons_4744" # beacon table name
tableRt = "beacons_4744_rt"

# Connect to Beacon Data
setRedshiftEndpoint(dsn)
setTable(table)
setTable(tableRt, tableType = "RESOURCE_TABLE")
;

Redshift endpoint set: 2017-11-30T16:17:58


In [2]:
# Time & Structures First
include("../../Lib/Include-Package-v2.1.jl")
;

In [3]:
#TV = timeVariables(2017,10,27,23,59,2017,11,3,23,59)
TV = weeklyTimeVariables(days=2)
#TV = yesterdayTimeVariables()
;

November 27, 2017 23:59:00 to November 29, 2017 23:59:00 Local Time
November 28, 2017 04:59:00 to November 30, 2017 04:59:00 UTC Time


In [4]:
UP = UrlParamsInit()
UP.agentOs = "%"
UP.deviceType = "%"
UP.limitRows = 10
#UP.limitRows = 250
UP.orderBy = "time"
UP.pageGroup = "%"   #productPageGroup
UP.samplesMin = 10
UP.sizeMin = 20000
UP.timeLowerMs = 2000.0
UP.timeUpperMs = 60000.0
UP.urlRegEx = "%"   #localUrl
UP.urlFull = "%"
UP.usePageLoad=false
UP.btView = "$(table)_all_dt_view_prod" #localtable
UP.rtView = "$(tableRt)_all_dt_view_prod" #localtableRt
UrlParamsValidate(UP)

SP = ShowParamsInit()
SP.desktop=false
SP.mobile=false
SP.devView=false
SP.criticalPathOnly=true
SP.debug=true
SP.debugLevel = 10   # Tests use even numbers with > tests, make this an odd number or zero
ShowParamsValidate(SP)
;

In [5]:
toppageurl = query("""\
    select 
        'None' as urlpagegroup,
        CASE WHEN (position('?' in url) > 0) then trim('/' from (substring(url for position('?' in substring(url from 9)) +7))) else trim('/' from url) end as urlgroup
    FROM $(UP.resourceTable)
    where
        "timestamp" between $(TV.startTimeMsUTC) and $(TV.endTimeMsUTC)
    group by urlgroup,urlpagegroup
    limit $(UP.limitRows)
 """);    

In [6]:
display(size(toppageurl))

(10,2)

In [7]:
#save for debug
toppageurlbackup = deepcopy(toppageurl);        

In [8]:
include("../../Lib/URL-Classification-Package-v2.0.jl")

WellKnownHostDirectory = wellKnownHostEncyclopedia(SP.debug);
WellKnownHost = wellKnownHostDictionary(SP.debug);
WellKnownPath = wellKnownPathDictionary(SP.debug);


Using Short Dictionary for Debug


In [10]:
include("../../Lib/URL-Classification-Package-v2.0.jl")

# Debug
toppageurl = deepcopy(toppageurlbackup)

scrubUrlToPrint(toppageurl);
classifyUrl(toppageurl,showProblems=false);        
#

z.moatads.com and Z
New Group NoneInner
cdn.krxd.net and C
New Group NoneInner
        ("cdn.krxd.net","cdn.krxd.net"),
www.facebook.com and W
New Group NoneInner
        ("www.facebook.com","www.facebook.com"),
www.zergnet.com and W
New Group NoneInner
www.googletagmanager.com and W
New Group NoneInner
        ("www.googletagmanager.com","www.googletagmanager.com"),
news.nationalgeographic.com and N
New Group NoneInner
news.nationalgeographic.com and N
New Group NoneInner
news.nationalgeographic.com and N
New Group NoneInner
tpc.googlesyndication.com and T
New Group NoneInner
        ("tpc.googlesyndication.com","tpc.googlesyndication.com"),
tpc.googlesyndication.com and T
New Group NoneInner
        ("tpc.googlesyndication.com","tpc.googlesyndication.com"),


In [ ]:
display(toppageurl)